<a href="https://colab.research.google.com/github/bdoma97/diplomamunka/blob/main/All_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/InceptionResNetV2_Testing /mydrive
!ls /mydrive


Mounted at /content/gdrive
0  3  6  9  C	      E  H			    J  M	  O  R	U  X
1  4  7  A  char1.h5  F  I			    K  N	  P  S	V  Y
2  5  8  B  D	      G  InceptionResNetV2_Testing  L  number.h5  Q  T	W  Z


In [2]:
#!pip install -q tensorflow==2.0.0b1
!pip install -q tensorflow==2.2
import tensorflow as tf


     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 460kB 54.7MB/s 
     |████████████████████████████████| 3.0MB 54.0MB/s 


In [3]:
print("\nTensorflow Version: " + tf.__version__)
!nvidia-smi -L


Tensorflow Version: 2.2.0
GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-b7750d12-b0af-4a4d-054c-1b6b77aa1855)


In [4]:
import keras
from tensorflow.keras.models import load_model

model_ = keras.models.load_model("gdrive/MyDrive/InceptionResNetV2_Testing/number.h5")
model_.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_406 (Conv2D)             (None, 37, 37, 32)   864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 37, 37, 32)   96          conv2d_406[0][0]                 
__________________________________________________________________________________________________
activation_406 (Activation)     (None, 37, 37, 32)   0           batch_normalization_406[0][0]    
____________________________________________________________________________________________

In [5]:
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans

import cleverhans

  Cloning https://github.com/tensorflow/cleverhans.git to /tmp/pip-install-788gr8vp/cleverhans
  Running command git clone -q https://github.com/tensorflow/cleverhans.git /tmp/pip-install-788gr8vp/cleverhans
     |████████████████████████████████| 163kB 5.7MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
  Created wheel for cleverhans: filename=cleverhans-4.0.0-cp37-none-any.whl size=92450 sha256=b4e41f283d788077015235a9e46ea27f491f8b6e87c1feae6d4e385454769f0e
  Stored in directory: /tmp/pip-ephem-wheel-cache-8owu5cqu/wheels/6e/59/ec/723a6f654aaf62c8c40f0f0850fdf71a4948598697f56c3bfa
Successfully built cleverhans


In [6]:
logits_model = tf.keras.Model(model_.input,model_.layers[-1].output)
logits_model

0-s jelzés

In [10]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/0/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 0:
    bad += 1
  counter += 1
  data.append([0, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_0_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/0/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 0:
      bad_prediction += 1
    image_counter += 1
    data.append([0, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_0_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/0/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 0:
      bad_prediction += 1
    image_counter += 1
    data.append([0, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_0_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/0/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 0:
    bad_prediction += 1
  counter += 1
  data.append([0, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_0_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

690
0 50 
Predikció ideje: 4.948645354000291
Rossz osztályozás: 8
690
0 50 
Epsilon: 0.001, rossz osztályozás: 17
Idő támadás képekre: 31.073586050999836
0 50 
Epsilon: 0.01, rossz osztályozás: 47
Idő támadás képekre: 31.04293155199997
0 50 
Epsilon: 0.1, rossz osztályozás: 51
Idő támadás képekre: 30.962570540999764
Kepfeldolgozás idő: 15.572319620000599
0 50 
Epsilon: 0.001, rossz osztályozás: 19
Idő támadás képekre: 40.59066248499994
0 50 
Epsilon: 0.01, rossz osztályozás: 80
Idő támadás képekre: 40.44377051200013
0 50 
Epsilon: 0.1, rossz osztályozás: 73
Idő támadás képekre: 40.80892761600035
Kepfeldolgozás idő: 15.33921714399969
690
0 
Epsilon:, rossz osztályozás: 6
Idő támadás képekre: 76.87655338599996
Kepfeldolgozás idő: 0.5140264529999286


1

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/1/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 1:
    bad += 1
  counter += 1
  data.append([1, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_1_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/1/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 1:
      bad_prediction += 1
    image_counter += 1
    data.append([1, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_1_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/1/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 1:
      bad_prediction += 1
    image_counter += 1
    data.append([1, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_1_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/1/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 1:
    bad_prediction += 1
  counter += 1
  data.append([1, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_1_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

2

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/2/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 2:
    bad += 1
  counter += 1
  data.append([2, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_2_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/2/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 2:
      bad_prediction += 1
    image_counter += 1
    data.append([2, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_2_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/2/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 2:
      bad_prediction += 1
    image_counter += 1
    data.append([2, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_2_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/2/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 2:
    bad_prediction += 1
  counter += 1
  data.append([2, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_2_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

3

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/3/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 3:
    bad += 1
  counter += 1
  data.append([3, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_3_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/3/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 3:
      bad_prediction += 1
    image_counter += 1
    data.append([3, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_3_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/3/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 3:
      bad_prediction += 1
    image_counter += 1
    data.append([3, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_3_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/3/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 3:
    bad_prediction += 1
  counter += 1
  data.append([3, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_3_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

4

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/4/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 4:
    bad += 1
  counter += 1
  data.append([4, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_4_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/4/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 4:
      bad_prediction += 1
    image_counter += 1
    data.append([4, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_4_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/4/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 4:
      bad_prediction += 1
    image_counter += 1
    data.append([4, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_4_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/4/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 4:
    bad_prediction += 1
  counter += 1
  data.append([4, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_4_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

5

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/5/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 5:
    bad += 1
  counter += 1
  data.append([5, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_5_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/5/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 5:
      bad_prediction += 1
    image_counter += 1
    data.append([5, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_5_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/5/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 5:
      bad_prediction += 1
    image_counter += 1
    data.append([5, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_5_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/5/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 5:
    bad_prediction += 1
  counter += 1
  data.append([5, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_5_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

6

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/6/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 6:
    bad += 1
  counter += 1
  data.append([6, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_6_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/6/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 6:
      bad_prediction += 1
    image_counter += 1
    data.append([6, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_6_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/6/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 6:
      bad_prediction += 1
    image_counter += 1
    data.append([6, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_6_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/6/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 6:
    bad_prediction += 1
  counter += 1
  data.append([6, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_6_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

7

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/7/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 7:
    bad += 1
  counter += 1
  data.append([7, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_7_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/7/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 7:
      bad_prediction += 1
    image_counter += 1
    data.append([7, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_7_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/7/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 7:
      bad_prediction += 1
    image_counter += 1
    data.append([7, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_7_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/7/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 7:
    bad_prediction += 1
  counter += 1
  data.append([7, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_7_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

8

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/8/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 8:
    bad += 1
  counter += 1
  data.append([8, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_8_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/8/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 8:
      bad_prediction += 1
    image_counter += 1
    data.append([8, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_8_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/8/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 8:
      bad_prediction += 1
    image_counter += 1
    data.append([8, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_8_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/8/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 8:
    bad_prediction += 1
  counter += 1
  data.append([8, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_8_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

9

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/9/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  if prediction != 9:
    bad += 1
  counter += 1
  data.append([9, prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_9_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/9/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 9:
      bad_prediction += 1
    image_counter += 1
    data.append([9, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2], adv_example_untargeted_label_pred[0][3],
                 adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5], adv_example_untargeted_label_pred[0][6],
                 adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_9_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
    'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/9/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    if new_prediction != 9:
      bad_prediction += 1
    image_counter += 1
    data.append([9, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
                 adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
                 adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_9_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/9/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  if new_prediction != 9:
    bad_prediction += 1
  counter += 1
  data.append([9, new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8], adv_example_untargeted_label_pred[0][9]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_9_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_0', 'Probability_1', 'Probability_2', 'Probability_3', 'Probability_4', 'Probability_5',
  'Probability_6', 'Probability_7', 'Probability_8', 'Probability_9'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

Betűk

In [14]:
import keras
from tensorflow.keras.models import load_model

model_ = keras.models.load_model("gdrive/MyDrive/InceptionResNetV2_Testing/char.h5")
model_.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 37, 37, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 37, 37, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 37, 37, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [15]:
logits_model = tf.keras.Model(model_.input,model_.layers[-1].output)
logits_model

In [16]:
def predict(number):
  if number == 0:
    return 'A'
  elif number == 1:
    return 'B'
  elif number == 2:
    return 'C'
  elif number == 3:
    return 'D'
  elif number == 4:
    return 'E'
  elif number == 5:
    return 'F'
  elif number == 6:
    return 'G'
  elif number == 7:
    return 'H'
  elif number == 8:
    return 'I'
  elif number == 9:
    return 'J'
  elif number == 10:
    return 'K'
  elif number == 11:
    return 'L'
  elif number == 12:
    return 'M'
  elif number == 13:
    return 'N'
  elif number == 14:
    return 'O'
  elif number == 15:
    return 'P'
  elif number == 16:
    return 'Q'
  elif number == 17:
    return 'R'
  elif number == 18:
    return 'S'
  elif number == 19:
    return 'T'
  elif number == 20:
    return 'U'
  elif number == 21:
    return 'V'
  elif number == 22:
    return 'W'
  elif number == 23:
    return 'X'
  elif number == 24:
    return 'Y'
  else:
    return 'Z'

A

In [18]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/A/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'A':
    bad += 1
  counter += 1
  data.append(['A', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_A_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/A/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'A':
      bad_prediction += 1
    image_counter += 1
    data.append(['A', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_A_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/A/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'A':
      bad_prediction += 1
    image_counter += 1
    data.append(['A', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_A_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/A/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'A':
    bad_prediction += 1
  counter += 1
  data.append(['A', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_A_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

1530
0 50 
Predikció ideje: 5.201243826000791
Rossz osztályozás: 0
1530
0 50 
Epsilon: 0.001, rossz osztályozás: 11
Idő támadás képekre: 30.9167491749987
0 50 
Epsilon: 0.01, rossz osztályozás: 36
Idő támadás képekre: 30.77771749200008
0 50 
Epsilon: 0.1, rossz osztályozás: 35
Idő támadás képekre: 30.92160282100008
Kepfeldolgozás idő: 15.898776321000128
0 50 
Epsilon: 0.001, rossz osztályozás: 13
Idő támadás képekre: 43.56634042600092
0 50 
Epsilon: 0.01, rossz osztályozás: 83
Idő támadás képekre: 40.751517971000794
0 50 
Epsilon: 0.1, rossz osztályozás: 65
Idő támadás képekre: 40.6418978460008
Kepfeldolgozás idő: 15.476837556000419
1530
0 
Epsilon:, rossz osztályozás: 10
Idő támadás képekre: 81.3018273250002
Kepfeldolgozás idő: 0.5158810760001415


B

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/B/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'B':
    bad += 1
  counter += 1
  data.append(['B', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_B_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/B/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'B':
      bad_prediction += 1
    image_counter += 1
    data.append(['B', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_B_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/B/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'B':
      bad_prediction += 1
    image_counter += 1
    data.append(['B', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_B_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/B/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'B':
    bad_prediction += 1
  counter += 1
  data.append(['B', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_B_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

C

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/C/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'C':
    bad += 1
  counter += 1
  data.append(['C', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_C_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/C/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'C':
      bad_prediction += 1
    image_counter += 1
    data.append(['C', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_C_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/C/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'C':
      bad_prediction += 1
    image_counter += 1
    data.append(['C', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_C_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/C/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'C':
    bad_prediction += 1
  counter += 1
  data.append(['C', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_C_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

D

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/D/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'D':
    bad += 1
  counter += 1
  data.append(['D', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_D_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/D/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'D':
      bad_prediction += 1
    image_counter += 1
    data.append(['D', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_D_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/D/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'D':
      bad_prediction += 1
    image_counter += 1
    data.append(['D', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_D_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/D/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'D':
    bad_prediction += 1
  counter += 1
  data.append(['D', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_D_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

E

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/E/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'E':
    bad += 1
  counter += 1
  data.append(['E', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_E_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/E/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'E':
      bad_prediction += 1
    image_counter += 1
    data.append(['E', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_E_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/E/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'E':
      bad_prediction += 1
    image_counter += 1
    data.append(['E', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_E_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/E/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'E':
    bad_prediction += 1
  counter += 1
  data.append(['E', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_E_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

F

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/F/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'F':
    bad += 1
  counter += 1
  data.append(['F', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_F_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/F/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'F':
      bad_prediction += 1
    image_counter += 1
    data.append(['F', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_F_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/F/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'F':
      bad_prediction += 1
    image_counter += 1
    data.append(['F', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_F_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/F/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'F':
    bad_prediction += 1
  counter += 1
  data.append(['F', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_F_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

G

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/G/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'G':
    bad += 1
  counter += 1
  data.append(['G', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_G_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/G/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'G':
      bad_prediction += 1
    image_counter += 1
    data.append(['G', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_G_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/G/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'G':
      bad_prediction += 1
    image_counter += 1
    data.append(['G', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_G_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/G/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'G':
    bad_prediction += 1
  counter += 1
  data.append(['G', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_G_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

H

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/H/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'H':
    bad += 1
  counter += 1
  data.append(['H', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_H_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/H/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'H':
      bad_prediction += 1
    image_counter += 1
    data.append(['H', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_H_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/H/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'H':
      bad_prediction += 1
    image_counter += 1
    data.append(['H', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_H_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/H/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'H':
    bad_prediction += 1
  counter += 1
  data.append(['H', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_H_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

I

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/I/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'I':
    bad += 1
  counter += 1
  data.append(['I', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_I_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/I/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'I':
      bad_prediction += 1
    image_counter += 1
    data.append(['I', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_I_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/I/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'I':
      bad_prediction += 1
    image_counter += 1
    data.append(['I', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_I_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/I/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'I':
    bad_prediction += 1
  counter += 1
  data.append(['I', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_I_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

J

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/J/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'J':
    bad += 1
  counter += 1
  data.append(['J', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_J_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/J/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'J':
      bad_prediction += 1
    image_counter += 1
    data.append(['J', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_J_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/J/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'J':
      bad_prediction += 1
    image_counter += 1
    data.append(['J', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_J_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/J/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'J':
    bad_prediction += 1
  counter += 1
  data.append(['J', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_J_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

K

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/K/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'K':
    bad += 1
  counter += 1
  data.append(['K', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_K_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/K/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'K':
      bad_prediction += 1
    image_counter += 1
    data.append(['K', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_K_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/K/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'K':
      bad_prediction += 1
    image_counter += 1
    data.append(['K', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_K_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/K/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'K':
    bad_prediction += 1
  counter += 1
  data.append(['K', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_K_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

L

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/L/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'L':
    bad += 1
  counter += 1
  data.append(['L', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_L_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/L/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'L':
      bad_prediction += 1
    image_counter += 1
    data.append(['L', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_L_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/L/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'L':
      bad_prediction += 1
    image_counter += 1
    data.append(['L', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_L_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/L/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'L':
    bad_prediction += 1
  counter += 1
  data.append(['L', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_L_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

M

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/M/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'M':
    bad += 1
  counter += 1
  data.append(['M', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_M_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/M/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'M':
      bad_prediction += 1
    image_counter += 1
    data.append(['M', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_M_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/M/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'M':
      bad_prediction += 1
    image_counter += 1
    data.append(['M', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_M_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/M/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'M':
    bad_prediction += 1
  counter += 1
  data.append(['M', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_M_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

N

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/N/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'N':
    bad += 1
  counter += 1
  data.append(['N', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_N_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/N/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'N':
      bad_prediction += 1
    image_counter += 1
    data.append(['N', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_N_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/N/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'N':
      bad_prediction += 1
    image_counter += 1
    data.append(['N', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_N_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/N/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'N':
    bad_prediction += 1
  counter += 1
  data.append(['N', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_N_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

O

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/O/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'O':
    bad += 1
  counter += 1
  data.append(['O', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_O_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/O/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'O':
      bad_prediction += 1
    image_counter += 1
    data.append(['O', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_O_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/O/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'O':
      bad_prediction += 1
    image_counter += 1
    data.append(['O', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_O_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/O/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'O':
    bad_prediction += 1
  counter += 1
  data.append(['O', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_O_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

P

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/P/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'P':
    bad += 1
  counter += 1
  data.append(['P', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_P_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/P/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'P':
      bad_prediction += 1
    image_counter += 1
    data.append(['P', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_P_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/P/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'P':
      bad_prediction += 1
    image_counter += 1
    data.append(['P', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_P_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/P/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'P':
    bad_prediction += 1
  counter += 1
  data.append(['P', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_P_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

Q

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Q/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'Q':
    bad += 1
  counter += 1
  data.append(['Q', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Q_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Q/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'Q':
      bad_prediction += 1
    image_counter += 1
    data.append(['Q', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Q_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Q/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'Q':
      bad_prediction += 1
    image_counter += 1
    data.append(['Q', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Q_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Q/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'Q':
    bad_prediction += 1
  counter += 1
  data.append(['Q', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Q_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

R

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/R/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'R':
    bad += 1
  counter += 1
  data.append(['R', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_R_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/R/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'R':
      bad_prediction += 1
    image_counter += 1
    data.append(['R', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_R_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/R/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'R':
      bad_prediction += 1
    image_counter += 1
    data.append(['R', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_R_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/R/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'R':
    bad_prediction += 1
  counter += 1
  data.append(['R', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_R_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

S

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/S/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'S':
    bad += 1
  counter += 1
  data.append(['S', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_S_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/S/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'S':
      bad_prediction += 1
    image_counter += 1
    data.append(['S', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_S_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/S/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'S':
      bad_prediction += 1
    image_counter += 1
    data.append(['S', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_S_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/S/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'S':
    bad_prediction += 1
  counter += 1
  data.append(['S', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_S_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

T

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/T/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'T':
    bad += 1
  counter += 1
  data.append(['T', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_T_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/T/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'T':
      bad_prediction += 1
    image_counter += 1
    data.append(['T', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_T_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/T/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'T':
      bad_prediction += 1
    image_counter += 1
    data.append(['T', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_T_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/T/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'T':
    bad_prediction += 1
  counter += 1
  data.append(['T', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_T_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

U

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/U/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'U':
    bad += 1
  counter += 1
  data.append(['U', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_U_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/U/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'U':
      bad_prediction += 1
    image_counter += 1
    data.append(['U', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_U_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/U/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'U':
      bad_prediction += 1
    image_counter += 1
    data.append(['U', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_U_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/U/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'U':
    bad_prediction += 1
  counter += 1
  data.append(['U', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_U_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

V

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/V/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'V':
    bad += 1
  counter += 1
  data.append(['V', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_V_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/V/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'V':
      bad_prediction += 1
    image_counter += 1
    data.append(['V', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_V_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/V/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'V':
      bad_prediction += 1
    image_counter += 1
    data.append(['V', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_V_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/V/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'V':
    bad_prediction += 1
  counter += 1
  data.append(['V', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_V_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

W

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/W/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'W':
    bad += 1
  counter += 1
  data.append(['W', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_W_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/W/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'W':
      bad_prediction += 1
    image_counter += 1
    data.append(['W', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_W_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/W/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'W':
      bad_prediction += 1
    image_counter += 1
    data.append(['W', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_W_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/W/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'W':
    bad_prediction += 1
  counter += 1
  data.append(['W', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_W_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

X

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/X/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'X':
    bad += 1
  counter += 1
  data.append(['X', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_X_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/X/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'X':
      bad_prediction += 1
    image_counter += 1
    data.append(['X', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_X_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/X/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'X':
      bad_prediction += 1
    image_counter += 1
    data.append(['X', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_X_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/X/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'X':
    bad_prediction += 1
  counter += 1
  data.append(['X', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_X_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

Y

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Y/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'Y':
    bad += 1
  counter += 1
  data.append(['Y', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Y_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Y/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'Y':
      bad_prediction += 1
    image_counter += 1
    data.append(['Y', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Y_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Y/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'Y':
      bad_prediction += 1
    image_counter += 1
    data.append(['Y', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Y_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Y/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'Y':
    bad_prediction += 1
  counter += 1
  data.append(['Y', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Y_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

Z

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.applications.inception_resnet_v2 import preprocess_input
import time
import csv

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Z/*")

bad = 0
counter = 0
timer = 0
data = []
print(len(images))
for i in images:
  if counter%50 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)
  t = time.process_time()
  yhat = model_.predict(image)
  elapsed_time = time.process_time() - t
  timer += elapsed_time
  prediction = np.argmax(yhat)
  prediction = predict(prediction)
  if prediction != 'Z':
    bad += 1
  counter += 1
  data.append(['Z', prediction, yhat[0][0], yhat[0][1], yhat[0][2], yhat[0][3], yhat[0][4], yhat[0][5], yhat[0][6], yhat[0][7], yhat[0][8], yhat[0][9],
               yhat[0][10], yhat[0][11], yhat[0][12], yhat[0][13], yhat[0][14], yhat[0][15], yhat[0][16], yhat[0][17], yhat[0][18], yhat[0][19], yhat[0][20],
               yhat[0][21], yhat[0][22], yhat[0][23], yhat[0][24], yhat[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Z_eredeti.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Predikció ideje: {}".format(timer))
print("Rossz osztályozás: {}".format(bad))

#FGSM
from cleverhans.tf2.attacks import fast_gradient_method
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Z/*")
print(len(images))
epsilon = [0.001, 0.01, 0.1]
timer_predict = 0
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end= " ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    t = time.process_time()
    adv_example_untargeted_label = fast_gradient_method.fast_gradient_method(logits_model, image, i, np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time

    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time
    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'Z':
      bad_prediction += 1
    image_counter += 1
    data.append(['Z', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Z_fgsm_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)
  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#BIM
from cleverhans.tf2.attacks import projected_gradient_descent
import time

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Z/*")
timer_predict = 0
epsilon = [0.001, 0.01, 0.1]
for i in epsilon:
  timer_attack = 0
  bad_prediction = 0
  image_counter = 0
  data = []
  for j in images:
    if image_counter%50 == 0:
      print(image_counter, end=" ")
    image = load_img(j, target_size=(75,75))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    t = time.process_time()
    adv_example_untargeted_label = projected_gradient_descent.projected_gradient_descent(logits_model, image, eps=i, eps_iter=i, nb_iter=3, norm=np.inf, targeted=False)
    elapsed_time = time.process_time() - t
    timer_attack += elapsed_time
    
    k = time.process_time()
    adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
    elapsed_time = time.process_time() - k
    timer_predict += elapsed_time

    new_prediction = np.argmax(adv_example_untargeted_label_pred)
    new_prediction = predict(new_prediction)
    if new_prediction != 'Z':
      bad_prediction += 1
    image_counter += 1
    data.append(['Z', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])
  with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Z_bim_' + str(i) + '.csv', mode='w') as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
    for k in data:
      writer.writerow(k)

  print()
  print("Epsilon: {}, rossz osztályozás: {}".format(i, bad_prediction))
  print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))

#CW2
import time
from cleverhans.tf2.attacks import carlini_wagner_l2

images = glob.glob("gdrive/MyDrive/InceptionResNetV2_Testing/Z/*")

timer_predict = 0
timer_attack = 0
bad_prediction = 0
counter = 0
data = []
print(len(images))
for i in images:
  if counter % 10 == 0:
    print(counter, end=' ')
  image = load_img(i, target_size=(75,75))
  image = img_to_array(image)
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  image = preprocess_input(image)

  t = time.process_time()
  adv_example_untargeted_label = carlini_wagner_l2.carlini_wagner_l2(logits_model, image, clip_min = -1.0, clip_max=1.0,
                                                                     batch_size=32, max_iterations=10, binary_search_steps=3, learning_rate=0.001)
  elapsed_time = time.process_time() - t
  timer_attack += elapsed_time
  
  k = time.process_time()
  adv_example_untargeted_label_pred = model_.predict(adv_example_untargeted_label)
  elapsed_time = time.process_time() - k
  timer_predict += elapsed_time

  new_prediction = np.argmax(adv_example_untargeted_label_pred)
  new_prediction = predict(new_prediction)
  if new_prediction != 'Z':
    bad_prediction += 1
  counter += 1
  data.append(['Z', new_prediction, adv_example_untargeted_label_pred[0][0], adv_example_untargeted_label_pred[0][1], adv_example_untargeted_label_pred[0][2],
               adv_example_untargeted_label_pred[0][3], adv_example_untargeted_label_pred[0][4], adv_example_untargeted_label_pred[0][5],
               adv_example_untargeted_label_pred[0][6], adv_example_untargeted_label_pred[0][7], adv_example_untargeted_label_pred[0][8],
               adv_example_untargeted_label_pred[0][9], adv_example_untargeted_label_pred[0][10], adv_example_untargeted_label_pred[0][11],
               adv_example_untargeted_label_pred[0][12], adv_example_untargeted_label_pred[0][13], adv_example_untargeted_label_pred[0][14],
               adv_example_untargeted_label_pred[0][15], adv_example_untargeted_label_pred[0][16], adv_example_untargeted_label_pred[0][17],
               adv_example_untargeted_label_pred[0][18], adv_example_untargeted_label_pred[0][19], adv_example_untargeted_label_pred[0][20],
               adv_example_untargeted_label_pred[0][21], adv_example_untargeted_label_pred[0][22], adv_example_untargeted_label_pred[0][23],
               adv_example_untargeted_label_pred[0][24], adv_example_untargeted_label_pred[0][25]])

with open('gdrive/MyDrive/InceptionResNetV2_Testing/predicition_Z_cw2.csv', mode='w') as f:
  writer = csv.writer(f, delimiter=';')
  writer.writerow(['Original', 'Predicted value', 'Probability_A', 'Probability_B', 'Probability_C', 'Probability_D', 'Probability_E', 'Probability_F',
  'Probability_G', 'Probability_H', 'Probability_I', 'Probability_J', 'Probability_K', 'Probability_L', 'Probability_M', 'Probability_N', 'Probability_O',
  'Probability_P', 'Probability_Q', 'Probability_R', 'Probability_S', 'Probability_T', 'Probability_U', 'Probability_V', 'Probability_W', 'Probability_X',
  'Probability_Y','Probability_Z'])
  for i in data:
    writer.writerow(i)
print()
print("Epsilon:, rossz osztályozás: {}".format(bad_prediction))
print("Idő támadás képekre: {}".format(timer_attack))
print("Kepfeldolgozás idő: {}".format(timer_predict))